# 1.3 AirSim 无人机基本控制

AirSim SDK（软件开发工具包）为无人机仿真提供了丰富的编程接口，支持通过代码实现无人机的基本控制、传感器数据获取及环境交互。以下是其核心功能的简要描述：

### 1. **连接与初始化控制**
   - **模拟器连接**：通过`MultirotorClient`类建立与AirSim模拟器的连接，确认通信状态后接管控制权。
   - **解锁与启动**：使用`enableApiControl(True)`和`armDisarm(True)`方法解锁无人机，使其进入可操作状态。

### 2. **基本飞行控制**
   - **起飞与悬停**：通过`takeoffAsync().join()`实现起飞，`hoverAsync().join()`保持悬停。
   - **路径规划**：调用`moveToPositionAsync(x, y, z, speed)`控制无人机向指定坐标移动，支持速度参数调整。
   - **返航与复位**：使用`reset()`方法重置无人机位置，`goHomeAsync()`可返回初始点。

### 3. **传感器与状态获取**
   - **传感器数据**：通过API获取IMU、GPS、气压计等传感器的实时数据（如`getImuData()`、`getGpsData()`）。
   - **状态监控**：`getMultirotorState()`返回无人机的实时位置、速度、姿态等状态信息。

### 4. **高级功能扩展**
   - **多机协同**：通过配置文件定义多台无人机，并在代码中按名称分别控制。
   - **环境交互**：支持动态调整天气（如雨雪效果）、光照条件，或通过API控制场景中的物体属性。
   - **ROS集成**：提供ROS/ROS2接口，便于与机器人操作系统联动，扩展复杂任务。

### 5. **SDK设计特点**
   - **跨平台支持**：兼容Python、C++、C#等语言，适用于Windows、Linux、macOS系统。
   - **模块化架构**：基于RPC通信（如msgpack-rpc），实现仿真引擎与外部控制逻辑的解耦。
   - **物理与视觉仿真融合**：结合虚幻引擎的渲染能力和自定义物理模型，提供高逼真度的仿真环境。

通过上述接口，开发者可快速构建无人机控制算法测试平台，同时结合AirSim的逼真场景和传感器模拟，验证视觉导航、自主避障等复杂功能。

本节主要讲airsim的初始化和基本飞行控制

In [1]:
!pip install numpy

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 8.4 MB/s eta 0:00:02
   --------------------------------- ------ 10.7/12.9 MB 33.5 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 33.8 MB/s eta 0:00:00


In [2]:
import sys
sys.path.append('../external-libraries')
import airsim
import math
import numpy as np

## 起飞降落

In [3]:
# connect to the AirSim simulator
#client = airsim.MultirotorClient(ip="192.168.31.194") #ip不写就是本地，写了就是跑airsim的机器
client = airsim.MultirotorClient() #ip不写就是本地，写了就是跑airsim的机器
client.confirmConnection()

# get control,允许api控制，默认是不允许的
client.enableApiControl(True)

# unlock，解锁，流程和真实飞行器一样
client.armDisarm(True)

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



True

In [4]:
# takeoff，起飞，很多无人机或者汽车控制的函数都有 Async/异步作为后缀，执行的时候会立即返回，不阻塞
# 但命令本身还是顺序执行的，所以需要join等待命令执行完毕，上一个命令执行完成，才会执行下一个命令
# 也可以不join，直接执行下一个命令，但是这样的话，就不知道上一个命令是否执行完成了，需要自己控制
client.takeoffAsync().join()
print("起飞-------------------------")

起飞-------------------------


In [5]:
#降落
client.landAsync().join()
print("降落-------------------------")



# lock
client.armDisarm(False)

# release control
client.enableApiControl(False)

降落-------------------------


## 定点飞行-offboard

In [9]:
# connect to the AirSim simulator
client = airsim.MultirotorClient()
client.confirmConnection()

# get control,允许api控制，默认是不允许的
client.enableApiControl(True)

# unlock，解锁，流程和真实飞行器一样
client.armDisarm(True)


# takeoff，起飞，很多无人机或者汽车控制的函数都有 Async/异步作为后缀，执行的时候会立即返回，不阻塞
# 但命令本身还是顺序执行的，所以需要join等待命令执行完毕，上一个命令执行完成，才会执行下一个命令
# 也可以不join，直接执行下一个命令，但是这样的话，就不知道上一个命令是否执行完成了，需要自己控制
client.takeoffAsync().join()
print("起飞--------------------------------")

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

起飞--------------------------------


In [10]:
#moveToZAsync，坐标系是NED，所以正数是向上，负数是向下
client.moveToZAsync(-3, 1).join()   # 上升到3米高度，1是速度

In [11]:
# 初始化4个点的坐标，并在视口中标识出来
points = [airsim.Vector3r(5, 0, -3),
          airsim.Vector3r(5, 5, -3),
          airsim.Vector3r(0, 5, -3),
          airsim.Vector3r(0, 0, -3)]
client.simPlotPoints(points, color_rgba=[0, 1, 0, 1], size=30, is_persistent=True)

In [12]:
# 方法1：按照逐个点飞，形成正方形
client.moveToPositionAsync(5, 0, -3, 1).join()  # 移动到(5,0,-3)的位置，1是速度x, y, z
client.moveToPositionAsync(5, 5, -3, 1).join()
client.moveToPositionAsync(0, 5, -3, 1).join()
client.moveToPositionAsync(0, 0, -3, 1).join()


In [13]:
# 方法2：直接按照航路点飞正方形轨迹
client.moveOnPathAsync(points, 1)

In [14]:
#降落
client.landAsync().join()
print("降落----------------------------------------------")


# lock
client.armDisarm(False)

# release control
client.enableApiControl(False)

降落----------------------------------------------


## 无人机的状态

在AirSim中，`simGetGroundTruthKinematics` 和 `simGetVehiclePose` 都是获取无人机状态的接口，但**返回数据的精度、用途和底层含义有本质区别**。以下是两者的详细对比：

---

### **1. `simGetVehiclePose()`**
- **功能**：获取无人机的**位姿（Pose）**，即位置和姿态。
- **返回数据**：
  ```python
  Pose(
      position=Vector3r(x, y, z),  # 位置（NED坐标系，单位：米）
      orientation=Quaternionr(w, x, y, z)  # 四元数姿态（相对世界坐标系）
  )
  ```
- **特点**：
  - 数据来源于仿真的**传感器模型**（如GPS、IMU），可能包含噪声。
  - 模拟真实无人机传感器输出的结果，适合测试SLAM、导航等算法。
  - 如果AirSim设置中启用了传感器噪声（默认关闭），返回的坐标会有扰动。

---

### **2. `simGetGroundTruthKinematics()`**
- **功能**：获取无人机的**真实运动学状态**（Ground Truth）。
- **返回数据**：
  ```python
  KinematicsState(
      position=Vector3r(x, y, z),  # 位置（绝对精确，NED坐标系）
      orientation=Quaternionr(w, x, y, z),  # 姿态（绝对精确）
      linear_velocity=Vector3r(vx, vy, vz),  # 线速度（米/秒）
      angular_velocity=Vector3r(wx, wy, wz),  # 角速度（弧度/秒）
      linear_acceleration=Vector3r(ax, ay, az),  # 线加速度（米/秒²）
      angular_acceleration=Vector3r(αx, αy, αz)  # 角加速度（弧度/秒²）
  )
  ```
- **特点**：
  - 数据来源于仿真的**物理引擎**，是绝对精确的“上帝视角”数据，无任何噪声。
  - 适合用于**算法验证**（如对比传感器数据与真实值的误差）或**控制算法设计**（如直接获取速度反馈）。

---

### **3. 关键区别总结**
| 特性                | `simGetVehiclePose()`              | `simGetGroundTruthKinematics()`    |
|--------------------|-----------------------------------|-----------------------------------|
| 数据来源             | 传感器模型（可能含噪声）            | 物理引擎（绝对精确）                |
| 数据维度             | 仅位置和姿态                      | 位置、姿态、速度、加速度            |
| 用途                | 模拟真实传感器输出                 | 算法验证、控制闭环设计              |
| 计算开销            | 低（仅读取传感器数据）              | 高（需从物理引擎获取完整状态）       |

---

### **4. 使用场景建议**
- **需要传感器级数据**（如测试SLAM、避障）：  
  使用 `simGetVehiclePose()`，并启用传感器噪声（通过AirSim的[设置文件](https://microsoft.github.io/AirSim/settings/)配置）。

- **需要真实状态验证**（如控制器调试、轨迹跟踪）：  
  使用 `simGetGroundTruthKinematics().position` 或 `linear_velocity`。

- **需要动力学参数**（如速度反馈、加速度前馈）：  
  直接读取 `linear_velocity` 和 `linear_acceleration`。



In [16]:
import time
import airsim

# connect to the AirSim simulator
client = airsim.MultirotorClient()
client.confirmConnection()

# get control,允许api控制，默认是不允许的
client.enableApiControl(True)

# unlock，解锁，流程和真实飞行器一样
client.armDisarm(True)


# takeoff，起飞，很多无人机或者汽车控制的函数都有 Async/异步作为后缀，执行的时候会立即返回，不阻塞
# 但命令本身还是顺序执行的，所以需要join等待命令执行完毕，上一个命令执行完成，才会执行下一个命令
# 也可以不join，直接执行下一个命令，但是这样的话，就不知道上一个命令是否执行完成了，需要自己控制
client.takeoffAsync().join()
print("起飞--------------------------------")


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

起飞--------------------------------


In [27]:
#moveToZAsync，坐标系是NED，所以正数是向上，负数是向下
client.moveToZAsync(-3, 1).join()   # 上升到3米高度，1是速度

for i in range(2):
    kinematic_state_groundtruth = client.simGetGroundTruthKinematics(vehicle_name='')
    #print("kinematic_state_groundtruth: ", kinematic_state_groundtruth)

    # state_groundtruth 的6个属性
    print(
    "position", kinematic_state_groundtruth.position,  # 位置信息
    "\n\n linear_velocity", kinematic_state_groundtruth.linear_velocity,  # 速度信息
    "\n\n linear_acceleration", kinematic_state_groundtruth.linear_acceleration , # 加速度信息
    "\n\n orientation", kinematic_state_groundtruth.orientation,  # 姿态信息
    "\n\n angular_velocity", kinematic_state_groundtruth.angular_velocity,  # 姿态角速率信息
    "\n\n angular_acceleration", kinematic_state_groundtruth.angular_acceleration,  # 姿态角加速度信息
    )

    # 无人机全局位置坐标真值
    x = kinematic_state_groundtruth.position.x_val  # 全局坐标系下，x轴方向的坐标
    y = kinematic_state_groundtruth.position.y_val  # 全局坐标系下，y轴方向的坐标
    z = kinematic_state_groundtruth.position.z_val  # 全局坐标系下，z轴方向的坐标
    print("x: ", x, "y: ", y, "z: ", z)

    time.sleep(1)
    print("-----------------------------------------------------------------------------------")

position <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': -2.6303207874298096} 

 linear_velocity <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': -3.4857908758567646e-05} 

 linear_acceleration <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': 0.0} 

 orientation <Quaternionr> {   'w_val': 0.9999998807907104,
    'x_val': 0.0,
    'y_val': 0.0,
    'z_val': 0.0} 

 angular_velocity <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': 0.0} 

 angular_acceleration <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': 0.0}
x:  0.0 y:  0.0 z:  -2.6303207874298096
-----------------------------------------------------------------------------------
position <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': -2.6303207874298096} 

 linear_velocity <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': -3.4857908758567646e-05} 

 linear_acceleration <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': 0.0} 

 orientation <Quater

In [30]:
pose = client.simGetVehiclePose()
print(pose)
print(pose.position.x_val, pose.position.y_val, pose.position.z_val)
print(pose.orientation)
print(airsim.to_eularian_angles(pose.orientation))

<Pose> {   'orientation': <Quaternionr> {   'w_val': 1.0,
    'x_val': -0.0,
    'y_val': 0.0,
    'z_val': 0.0},
    'position': <Vector3r> {   'x_val': 0.0,
    'y_val': 0.0,
    'z_val': -2.630319356918335}}
0.0 0.0 -2.630319356918335
<Quaternionr> {   'w_val': 1.0,
    'x_val': -0.0,
    'y_val': 0.0,
    'z_val': 0.0}
(0.0, 0.0, 0.0)


In [31]:
#降落
client.landAsync().join()
print("降落----------------------------------------------")



# lock
client.armDisarm(False)

# release control
client.enableApiControl(False)

降落----------------------------------------------
